<a href="https://colab.research.google.com/github/Raquel223/Ciclo-Diurno-Relampagos/blob/main/Tabela_distancia_BrasilDAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cálculo da distância do centro da rede de detecção de relâmpagos SPLMA e a coordenada de cada relâmpago**
---
- **OBJETIVO:** Filtrar os relâmpagos que ocorrem dentro do raio de 100 km do centro da SPLMA, pois é a área de melhor eficiência de detecção da rede.  

---

**Dados utilizados**: Os dados de relâmpagos que foram utilizados nesse código foram obtidos da rede BrasilDAT, através da campanha CHUVA-Vale do Paraíba. Neste exemplo foram utilizados dados do mês de dezembro.


---



- **METODOLOGIA:** Nessa aula realizaremos os seguintes procedimentos:

    1.   **Importação** das biblotecas
    2.   Montagem do **drive online**
    3.   **Abrir e extrair** o arquivo .tar
    4.   **Leitura e junção** dos arquivos
    5.   **Cálculo** da distância
    6.   **Filtro** da distância
    7.   **Salvar** a tabela
    
---

**OBSERVAÇÕES IMPORTANTES**: 
1. Fazer mudanças, veja em: **### >>> MUDANÇAS AQUI <<< ###**
    - 1. Caminho do seu drive 

---
- **Realizado por:** Raquel Gonçalves Pereira e Enrique V. Mattos - 25/08/2021


# **Importação das bibliotecas**

In [ ]:
import pandas as pd
import glob
import os
from matplotlib import pyplot as plt
import numpy as np
import  tarfile
import geopy.distance

# **Montagem do drive online**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **### >>> MUDANÇAS AQUI <<< ###**:
- Insira o **caminho + nome** do arquivo

In [ ]:
filename = '/content/drive/MyDrive/GitHub/BrasilDAT/2_dezembro.tar.gz'

dir = os.path.dirname(filename)

# **Abrir o arquivo .tar e extrair**



In [ ]:
tar = tarfile.open(filename)
tar.getnames()
tar.extractall()

# **Leitura de todos os arquivos**

*   Juntar todos os arquivos em apenas um dataframe




In [ ]:
lista=[]
for f in tar.getnames():
  df = pd.read_csv(f, delimiter='\s+', compression= 'gzip')
  lista.append(df)
todos_arquivos = pd.concat((lista), ignore_index = True)
print(todos_arquivos)


               id tipo   ano  mes  dia  hor  min        seg        lat  \
0        99999999   IC  2011   12    1    0    0   0.639879 -25.529369   
1        99999999   IC  2011   12    1    0    0   0.814873 -21.569209   
2        99999999   IC  2011   12    1    0    0   1.112399 -21.377371   
3        99999999   IC  2011   12    1    0    0   1.125562 -21.392892   
4        99999999   IC  2011   12    1    0    0   2.356357 -21.263056   
...           ...  ...   ...  ...  ...  ...  ...        ...        ...   
3569931  99999999   IC  2011   12   31   23   59  50.984325 -18.938961   
3569932  99999999   IC  2011   12   31   23   59  51.209832 -18.910619   
3569933  99999999   IC  2011   12   31   23   59  51.926988 -17.656357   
3569934  99999999   IC  2011   12   31   23   59  51.928405 -17.659146   
3569935  99999999   CG  2011   12   31   23   59  52.045696 -17.646229   

               lon  pc(A)  alt_ic(m)  sens  mult_certa  
0       -48.610768    3.2          0     0           0

**Mostrando como ficou o dataframe concatenado**

In [ ]:
display(todos_arquivos)
#shape mostrar quantas linha e colunas tem
#todos_arquivos.shape

,id,tipo,ano,mes,dia,hor,min,seg,lat,lon,pc(A),alt_ic(m),sens,mult_certa
0,99999999,IC,2011,12,1,0,0,0.639879,-25.529369,-48.610768,3.2,0,0,0
1,99999999,IC,2011,12,1,0,0,0.814873,-21.569209,-48.966423,8.5,0,0,0
2,99999999,IC,2011,12,1,0,0,1.112399,-21.377371,-43.787563,4.6,0,0,0
3,99999999,IC,2011,12,1,0,0,1.125562,-21.392892,-43.815152,4.9,0,0,0
4,99999999,IC,2011,12,1,0,0,2.356357,-21.263056,-46.292658,-3.4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569931,99999999,IC,2011,12,31,23,59,50.984325,-18.938961,-50.053127,6.3,0,0,0
3569932,99999999,IC,2011,12,31,23,59,51.209832,-18.910619,-50.031056,7.1,0,0,0
3569933,99999999,IC,2011,12,31,23,59,51.926988,-17.656357,-41.500538,11.9,0,0,0
3569934,99999999,IC,2011,12,31,23,59,51.928405,-17.659146,-41.504944,9.3,0,0,0


**Filtro das colunas que irão ser utilizadas**

In [ ]:
#filtrei só as colunas que quero
tipos_hora_pico = todos_arquivos[['tipo','ano','mes','dia','hor','pc(A)','lat','lon']]
display(tipos_hora_pico)
type(tipos_hora_pico)

,tipo,ano,mes,dia,hor,pc(A),lat,lon
0,IC,2011,12,1,0,3.2,-25.529369,-48.610768
1,IC,2011,12,1,0,8.5,-21.569209,-48.966423
2,IC,2011,12,1,0,4.6,-21.377371,-43.787563
3,IC,2011,12,1,0,4.9,-21.392892,-43.815152
4,IC,2011,12,1,0,-3.4,-21.263056,-46.292658
...,...,...,...,...,...,...,...,...
3569931,IC,2011,12,31,23,6.3,-18.938961,-50.053127
3569932,IC,2011,12,31,23,7.1,-18.910619,-50.031056
3569933,IC,2011,12,31,23,11.9,-17.656357,-41.500538
3569934,IC,2011,12,31,23,9.3,-17.659146,-41.504944


pandas.core.frame.DataFrame

# **Cálculo da distância do centro da rede SPLMA e coordenada de cada relâmpago**

In [ ]:
dist = []
for i in tipos_hora_pico.itertuples():
   coords_1 = (-23.5489740,-46.6388100)   #coordenadas do centro da rede SPLMA
   coords_2 = (i.lat, i.lon)              #coordenadas de cada relâmpago
   dist.append(geopy.distance.distance(coords_1, coords_2).km)   #cálculo da distância
tipos_hora_pico['distancia'] = dist  #criada uma coluna chamada "distancia" para inserir os valores da distancia do centro da rede e de cada relâmpago
tipos_hora_pico

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,tipo,ano,mes,dia,hor,pc(A),lat,lon,distancia
0,IC,2011,12,1,0,3.2,-25.529369,-48.610768,296.705523
1,IC,2011,12,1,0,8.5,-21.569209,-48.966423,324.602873
2,IC,2011,12,1,0,4.6,-21.377371,-43.787563,379.380437
3,IC,2011,12,1,0,4.9,-21.392892,-43.815152,376.083240
4,IC,2011,12,1,0,-3.4,-21.263056,-46.292658,255.629551
...,...,...,...,...,...,...,...,...,...
3569931,IC,2011,12,31,23,6.3,-18.938961,-50.053127,621.316466
3569932,IC,2011,12,31,23,7.1,-18.910619,-50.031056,622.616931
3569933,IC,2011,12,31,23,11.9,-17.656357,-41.500538,843.874716
3569934,IC,2011,12,31,23,9.3,-17.659146,-41.504944,843.342012


# **Filtro para pegar somente os relâmpagos em que a distância é igual ou menor que 100 km**

In [ ]:
final = tipos_hora_pico[tipos_hora_pico['distancia'] <= 100]
final

,tipo,ano,mes,dia,hor,pc(A),lat,lon,distancia
291,IC,2011,12,1,0,2.0,-23.831063,-47.362767,80.176647
393,IC,2011,12,1,0,-3.6,-23.747681,-47.299678,70.926538
427,IC,2011,12,1,0,-3.0,-23.680954,-47.207217,59.820518
464,IC,2011,12,1,0,2.2,-23.660856,-47.214331,60.030432
730,IC,2011,12,1,0,-2.0,-23.883682,-47.379116,84.102026
...,...,...,...,...,...,...,...,...,...
3565780,IC,2011,12,31,23,11.4,-23.180063,-47.192800,69.840121
3568857,IC,2011,12,31,23,2.4,-23.360032,-45.774964,90.710722
3568858,IC,2011,12,31,23,1.4,-23.373971,-45.744405,93.414592
3568859,IC,2011,12,31,23,-2.3,-23.365797,-45.753093,92.742952


# **Salvar a tabela**

In [ ]:
final.to_csv(f'{dir}/Novo_Dezembro.csv', index = True, header=True) 